<a href="https://colab.research.google.com/github/Janamawasy/ML/blob/main/titanic_comp_ANN_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
tf.__version__

'2.15.0'

In [2]:
# -------------- code - acc=0.85   score-0.76

train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (1).csv')
y_test_data = pd.read_csv('gender_submission.csv')


In [3]:
all_data = pd.concat([train, test], ignore_index=True)
all_data.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [ ]:
all_data = all_data.dropna(subset='Embarked')
all_data['Age'] = all_data['Age'].fillna(-1)
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())

# # split cabin to number and letter(floor)
all_data['Floor'] = all_data['Cabin'].str.extract('([a-zA-Z]+)', expand=False)
all_data['Room_Num'] = all_data['Cabin'].str.extract('(\d+)', expand=False)
all_data['Room_Num'] = all_data['Room_Num'].fillna(-1).astype(int)
all_data['Floor'] = all_data['Floor'].fillna(-1)
all_data = all_data.drop(columns=['Cabin', 'Name','PassengerId'])

all_data.isna().sum()



In [5]:
# label encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
all_data['Sex'] = le.fit_transform(all_data['Sex'])
all_data['Ticket'] = le.fit_transform(all_data['Ticket'])
# FLOOR NUMBER
category_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5,'F': 6,'G': 7,'T': 8, -1:-1}

# Use the map function to replace categories with numerical values
all_data['Floor'] = all_data['Floor'].map(category_mapping)


# One Hot Encoding
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Embarked'])], remainder='passthrough')
all_data = pd.DataFrame(ct.fit_transform(all_data))




In [6]:
# split the data - remove passenger id

Survived_col = all_data.columns[all_data.isna().any()].tolist()[0]

nan_indexes = np.where(np.isnan(all_data.iloc[:, Survived_col]))[0][0]
limit = nan_indexes
x_train = all_data.iloc[0:limit,:]
y_train = all_data.iloc[0:limit,Survived_col]

x_test = all_data.iloc[limit:,:]
y_test = all_data.iloc[limit:,Survived_col]

# remove Survived_col from x
x_test = x_test.drop(columns=Survived_col)
x_train = x_train.drop(columns=Survived_col)

In [7]:
# -------------------- CORR +  DELETING CORRELATED FEATURES
corr_data = all_data.drop(columns=3)

# Set the correlation threshold
correlation_threshold = 0.7

# Calculate the correlation matrix
correlation_matrix = corr_data.corr(numeric_only=True)

# Find features with correlation above the threshold
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= correlation_threshold:
            colname = correlation_matrix.columns[i]
            highly_correlated_features.add(colname)

print(np.array(highly_correlated_features))
print(correlation_matrix)

# indices_to_drop = [pd.DataFrame(corr_data).columns.get_loc(col) for col in highly_correlated_features]
# filtered_data = np.delete(corr_data, indices_to_drop, axis=1)
# correlation_matrix = pd.DataFrame(filtered_data).corr()
# print((abs(correlation_matrix) >= 0.7).sum().sum()-len(correlation_matrix) == 0)


{2, 12}
          0         1         2         4         5         6         7   \
0   1.000000 -0.164463 -0.778161 -0.271415 -0.067720  0.043929 -0.048788   
1  -0.164463  1.000000 -0.491534  0.230169 -0.089445 -0.227527 -0.048926   
2  -0.778161 -0.491534  1.000000  0.093071  0.116741  0.106091  0.074227   
4  -0.271415  0.230169  0.093071  1.000000  0.121826 -0.406491  0.059819   
5  -0.067720 -0.089445  0.116741  0.121826  1.000000  0.001314 -0.110768   
6   0.043929 -0.227527  0.106091 -0.406491  0.001314  1.000000 -0.131426   
7  -0.048788 -0.048926  0.074227  0.059819 -0.110768 -0.131426  1.000000   
8  -0.008985 -0.101186  0.072362  0.017304 -0.214371 -0.036558  0.373383   
9  -0.054576  0.059468  0.010319  0.307040  0.021692 -0.091241  0.062889   
10  0.287309 -0.129604 -0.171136 -0.557915 -0.184192  0.212115  0.161141   
11  0.198275 -0.128539 -0.093207 -0.604819 -0.134803  0.257548 -0.008675   
12  0.223376 -0.098652 -0.134399 -0.578487 -0.099004  0.242152 -0.002768   

   

In [9]:
# ------------ CORR BETWEEN X AND Y
# correlation_with_y = df.corrwith(np.array(y))

x_transposed = np.transpose(x_train)

# Compute the correlation between each feature and y
correlation_with_y = np.corrcoef(x_transposed, y_train)[0:-1, -1]
print(correlation_with_y)

[ 0.16996597  0.00453573 -0.15177705 -0.33554886 -0.54158492  0.00826467
 -0.03404     0.08315078 -0.16314171  0.25529046  0.29716829  0.23053655]


In [50]:
# ------------------------------ANN MODEL------------------------------------
from sklearn.model_selection import train_test_split

# scaling data - normilizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

#Training

#Init the ANN
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

# Actual training
ann.fit(x_train, y_train, batch_size=32, epochs=100)

y_pred = ann.predict(x_test)

from sklearn.metrics import multilabel_confusion_matrix,accuracy_score, confusion_matrix

y_pred_ANN = np.where(y_pred > 0.5, 1, 0)



Epoch 1/100
28/28 [==============================] - 2s 3ms/step - loss: 0.6562 - accuracy: 0.6592
Epoch 2/100
28/28 [==============================] - 0s 16ms/step - loss: 0.5976 - accuracy: 0.7199
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5573 - accuracy: 0.7402
Epoch 4/100
28/28 [==============================] - 0s 4ms/step - loss: 0.5285 - accuracy: 0.7514
Epoch 5/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5107 - accuracy: 0.7548
Epoch 6/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4971 - accuracy: 0.7638
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4881 - accuracy: 0.7660
Epoch 8/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4825 - accuracy: 0.7750
Epoch 9/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4768 - accuracy: 0.7784
Epoch 10/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4706 - accuracy: 0.7885
Epoch 11

In [51]:
y_test = y_test_data['Survived']

In [52]:
# Evaluate ANN model ::
cm_ANN = confusion_matrix(y_test, y_pred_ANN)
accuracy_ANN = accuracy_score(y_test, y_pred_ANN)

print('confusiom matrix:')
print(cm_ANN)
print('acc is:', accuracy_ANN)


confusiom matrix:
[[246  20]
 [ 35 117]]
acc is: 0.868421052631579


In [53]:
from sklearn.linear_model import LogisticRegression

# Initialize the logistic regression model
logistic_regression_model = LogisticRegression()

# Fit the model to the training data
logistic_regression_model.fit(x_train, y_train)

# Make predictions on the testing data
y_pred_reg = logistic_regression_model.predict(x_test)


In [54]:
# Evaluate Logistic Regression model ::
accuracy_reg = accuracy_score(y_test, y_pred_reg)
cm_reg = confusion_matrix(y_test,y_pred_reg)

print('confusiom matrix:')
print(cm_reg)
print("Accuracy:", accuracy_reg)


confusiom matrix:
[[255  11]
 [ 11 141]]
Accuracy: 0.9473684210526315


In [55]:
# download output files:
import csv

y_pred_reg = np.array([int(x) for x in y_pred_reg])

# Sample data
passenger_ids = test['PassengerId'].values

# Create a DataFrame from the given data
data1 = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': y_pred_ANN.flatten()  # Flatten the array to match the shape
})

data2 = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': y_pred_reg.flatten()  # Flatten the array to match the shape
})

# Specify the file path
csv_file_path1 = 'output_ANN.csv'
csv_file_path2 = 'output_reg.csv'

# Write the DataFrame to a CSV file
data1.to_csv(csv_file_path1, index=False)
data2.to_csv(csv_file_path2, index=False)

from google.colab import files

files.download('output_ANN.csv')
files.download('output_reg.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>